## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation, Dense
from keras.datasets import cifar10
from keras.layers import GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [9]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\asi\Anaconda3\envs\python36_tf19\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [12]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),padding='same',input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 5s 94us/sample - loss: 1.2975 - accuracy: 0.5461
Epoch 2/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.8594 - accuracy: 0.6969
Epoch 3/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.6293 - accuracy: 0.7784
Epoch 4/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.4357 - accuracy: 0.8489
Epoch 5/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.2893 - accuracy: 0.9008
Epoch 6/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.1905 - accuracy: 0.9353
Epoch 7/100
50000/50000 [==============================] - 4s 71us/sample - loss: 0.1479 - accuracy: 0.9485
Epoch 8/100
50000/50000 [==============================] - 4s 70us/sample - loss: 0.1137 - accuracy: 0.9612
Epoch 9/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.1120 - accuracy: 0.9615
Epoch

50000/50000 [==============================] - 4s 73us/sample - loss: 0.0096 - accuracy: 0.9968
Epoch 76/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.0090 - accuracy: 0.9971
Epoch 77/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.0216 - accuracy: 0.9938
Epoch 78/100
50000/50000 [==============================] - 4s 73us/sample - loss: 0.0210 - accuracy: 0.9937
Epoch 79/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.0104 - accuracy: 0.9967
Epoch 80/100
50000/50000 [==============================] - 4s 71us/sample - loss: 0.0089 - accuracy: 0.9972
Epoch 81/100
50000/50000 [==============================] - 4s 73us/sample - loss: 0.0093 - accuracy: 0.9967
Epoch 82/100
50000/50000 [==============================] - 4s 73us/sample - loss: 0.0110 - accuracy: 0.9969
Epoch 83/100
50000/50000 [==============================] - 4s 72us/sample - loss: 0.0125 - accuracy: 0.9963
Epoch 84/100
50000/50000 [======

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [32]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.3619592e-21, 0.0000000e+00, 7.7137704e-25, 6.8325937e-01,
        3.1674063e-01, 0.0000000e+00, 5.6022221e-26, 0.0000000e+00,
        1.6030348e-15, 0.0000000e+00]], dtype=float32)

In [11]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),padding='same',input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())


classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 5s 102us/sample - loss: 1.7042 - accuracy: 0.3723
Epoch 2/100
50000/50000 [==============================] - 5s 93us/sample - loss: 1.4153 - accuracy: 0.4892
Epoch 3/100
50000/50000 [==============================] - 5s 92us/sample - loss: 1.2794 - accuracy: 0.5415
Epoch 4/100
50000/50000 [==============================] - ETA: 0s - loss: 1.1979 - accuracy: 0.57 - 5s 92us/sample - loss: 1.1975 - accuracy: 0.5739
Epoch 5/100
50000/50000 [==============================] - 5s 93us/sample - loss: 1.1441 - accuracy: 0.5919
Epoch 6/100
50000/50000 [==============================] - 5s 93us/sample - loss: 1.1036 - accuracy: 0.6087
Epoch 7/100
50000/50000 [==============================] - 5s 93us/sample - loss: 1.0737 - accuracy: 0.6193
Epoch 8/100
50000/50000 [==============================] - 5s 93us/sample - loss: 1.0497 - accuracy: 0.6275
Epoch 9/100
50000/50000 [==============================] - 5s 93us/sam